In [1]:
import pandas as pd

# Load the original CSV file with the correct delimiter
file_path = '/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv'
df = pd.read_csv(file_path, delimiter='|')
df.head(10)

# Extract the 'comment' column
comment_column = df[' comment']

# Save the extracted column to a new CSV file
output_file_path = 'yey.csv'
comment_column.to_csv(output_file_path, index=False)
import pandas as pd
import tensorflow as tf

# Load the CSV file
file_path = 'yey.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Ensure all data in 'comment' column are strings and handle missing values
df[' comment'] = df[' comment'].astype(str).fillna('')

# Convert the DataFrame to a TensorFlow dataset
tf_dataset = tf.data.Dataset.from_tensor_slices(df[' comment'].values)

# Optionally, save the TensorFlow dataset as a TFRecord file
output_tfrecord_path = 'newten.tfrecord'  # Replace with your desired save path
with tf.io.TFRecordWriter(output_tfrecord_path) as writer:
    for comment in tf_dataset:
        feature = {
            'comment': tf.train.Feature(bytes_list=tf.train.BytesList(value=[comment.numpy()]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())

print(f"TFRecord file saved to: {output_tfrecord_path}")
import tensorflow as tf

# File path to your TFRecord file
file_path = 'newten.tfrecord'

# Create a TFRecordDataset
raw_dataset = tf.data.TFRecordDataset(file_path)

# Print the raw records to see what they look like
for raw_record in raw_dataset.take(5):  # Adjust the number of records to preview
    print(raw_record)
import tensorflow as tf

# Function to parse a single example from the TFRecord file
def _parse_function(proto):
    # Define the expected feature types
    keys_to_features = {
        'comment': tf.io.FixedLenFeature([], tf.string),  # The 'comment' field is of type string
    }
    # Parse the input tf.train.Example proto using the feature description
    parsed_features = tf.io.parse_single_example(proto, keys_to_features)
    return parsed_features


# File path to your TFRecord file
file_path = 'newten.tfrecord'  # Replace with your actual file path

# Create a TFRecordDataset
raw_dataset = tf.data.TFRecordDataset(file_path)

# Parse the dataset using the function defined above
parsed_dataset = raw_dataset.map(_parse_function)

# Iterate through the parsed dataset and print the 'comment' field
for record in parsed_dataset.take(5):  # Adjust the number to see more records
    comment = record['comment'].numpy().decode('utf-8')
    print(comment)
import tensorflow as tf

# Function to parse the TFRecord files
def _parse_function(proto):
    # Define your feature description dictionary based on the TFRecord structure
    feature_description = {
        'comment': tf.io.FixedLenFeature([], tf.string),
        # Add other features if necessary
    }
    # Parse the input tf.train.Example proto using the dictionary above
    return tf.io.parse_single_example(proto, feature_description)

# Load and parse the TFRecord dataset
dataset = tf.data.TFRecordDataset('newten.tfrecord')
dataset = dataset.map(_parse_function)
def extract_text(dataset):
    texts = []
    for record in dataset:
        texts.append(record['comment'].numpy().decode('utf-8'))
    return texts

# Extract text data
text_data = extract_text(dataset)
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

# Initialize a BPE tokenizer
tokenizer = Tokenizer(BPE())

# Set the pre-tokenizer to split on whitespace
tokenizer.pre_tokenizer = Whitespace()

# Initialize a BPE trainer with your desired vocabulary size
trainer = BpeTrainer(vocab_size=30000, special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])

# Train the tokenizer
tokenizer.train_from_iterator(text_data, trainer=trainer)
def tokenize_example(text_tensor):
    # Decode the text from the tensor
    text = text_tensor.numpy().decode('utf-8')
    # Tokenize the text using the BPE tokenizer
    encoding = tokenizer.encode(text)
    # Return the token ids as a TensorFlow tensor
    return tf.constant(encoding.ids, dtype=tf.int32)

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(
    lambda x: tf.py_function(tokenize_example, inp=[x['comment']], Tout=tf.int32)
)
for tokenized_text in tokenized_dataset.take(10):
    print(tokenized_text.numpy())
import tensorflow as tf

# Convert the TensorFlow dataset to a list of lists of tokens
def extract_tokenized_sentences(dataset):
    tokenized_sentences = []
    for tokenized_text in dataset:
        # Convert TensorFlow tensor to a list of integers
        tokenized_sentences.append(tokenized_text.numpy().tolist())
    return tokenized_sentences

# Extract tokenized sentences
# Note: This may take a while if the dataset is very large
tokenized_sentences = extract_tokenized_sentences(tokenized_dataset)

# Write the tokenized sentences to a file
file_path = 'tokenized_sentences.txt'  # Path to your output file

with open(file_path, 'w') as file:
    for sentence in tokenized_sentences:
        # Join tokens with space and write each sentence on a new line
        file.write(' '.join(map(str, sentence)) + '\n')
import tensorflow as tf
import numpy as np

# Load tokenized sentences from a text file
def load_tokenized_sentences(file_path):
    tokenized_sentences = []
    with open(file_path, 'r') as file:
        for line in file:
            tokens = list(map(int, line.strip().split()))
            tokenized_sentences.append(tokens)
    return tokenized_sentences

# Parameters
file_path = 'tokenized_sentences.txt'  # Replace with your file path
max_seq_length = 20  # Set based on your data
padding_value = 0
batch_size = 32
embedding_dim = 768
vocab_size = 20000  # Adjust based on your tokenizer's vocabulary size
oov_token = vocab_size  # Token index for out-of-vocabulary (OOV) tokens

# Load and pad sequences
tokenized_sentences = load_tokenized_sentences(file_path)

def pad_sequences(sequences, max_len, padding_value):
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
        sequences,
        maxlen=max_len,
        padding='post',
        value=padding_value
    )
    return padded_sequences

# Pad sequences to fixed length and handle OOV tokens
def process_tokens(sequences, vocab_size, oov_token):
    processed_sequences = []
    for seq in sequences:
        processed_seq = [token if token < vocab_size else oov_token for token in seq]
        processed_sequences.append(processed_seq)
    return pad_sequences(processed_sequences, max_seq_length, padding_value)

padded_sequences = process_tokens(tokenized_sentences, vocab_size, oov_token)

# Create a TensorFlow dataset from the padded sequences
dataset = tf.data.Dataset.from_tensor_slices(padded_sequences)

# Batch the dataset
batched_dataset = dataset.batch(batch_size, drop_remainder=True)

# Define the embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim)  # +1 for OOV token

# Function to get embeddings
def get_embeddings(tokens_batch):
    return embedding_layer(tokens_batch)

# Process each batch
for batch in batched_dataset:
    print("Processing a batch...")
    tokens_batch = batch  # TensorFlow tensors should be used directly
    embeddings_batch = get_embeddings(tokens_batch)
    print(f"Tokens batch shape: {tokens_batch.shape}")
    print(f"Embeddings batch shape: {embeddings_batch.shape}")
    # Handle the embeddings_batch as needed
import tensorflow as tf
import numpy as np

# Function to load tokenized sentences from a file
def load_tokenized_sentences(file_path):
    tokenized_sentences = []
    with open(file_path, 'r') as file:
        for line in file:
            tokens = list(map(int, line.strip().split()))
            tokenized_sentences.append(tokens)
    return tokenized_sentences

# Parameters
file_path = 'tokenized_sentences.txt'
max_seq_length = 20
padding_value = 0
embedding_dim = 768
vocab_size = 20000
oov_token = vocab_size

# Load and pad sequences
tokenized_sentences = load_tokenized_sentences(file_path)

def pad_sequences(sequences, max_len, padding_value):
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(
        sequences, maxlen=max_len, padding='post', value=padding_value)
    return padded_sequences

def process_tokens(sequences, vocab_size, oov_token):
    processed_sequences = []
    for seq in sequences:
        processed_seq = [token if token < vocab_size else oov_token for token in seq]
        processed_sequences.append(processed_seq)
    return pad_sequences(processed_sequences, max_seq_length, padding_value)

padded_sequences = process_tokens(tokenized_sentences, vocab_size, oov_token)

# Create a TensorFlow dataset from the padded sequences and batch them
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(padded_sequences).batch(batch_size)

# Define the embedding layer
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size + 1, output_dim=embedding_dim)

# Define the GlobalMaxPooling layer to reduce sequence length
pooling_layer = tf.keras.layers.GlobalMaxPooling1D()

# Function to get embeddings
def get_embeddings(tokens_batch):
    embeddings = embedding_layer(tokens_batch)
    pooled_embeddings = pooling_layer(embeddings)
    return pooled_embeddings

# Process the dataset in batches
common_embeddings = []
embeddings_list = []
for tokens_batch in dataset:
    embeddings_batch = get_embeddings(tokens_batch)
    embeddings_list.extend(embeddings_batch)

# Group every 5 embeddings together
for i in range(0, len(embeddings_list), 5):
    batch_embeddings = embeddings_list[i:i+5]
    common_embedding = tf.reduce_mean(batch_embeddings, axis=0)
    common_embeddings.append(common_embedding)

# Convert the list of common embeddings to a tensor
common_embeddings = tf.stack(common_embeddings)

print("Common Embeddings Shape:", common_embeddings.shape)

TFRecord file saved to: newten.tfrecord
tf.Tensor(b'\ne\nc\n\x07comment\x12X\nV\nT Two young guys with shaggy hair look at their hands while hanging out in the yard .', shape=(), dtype=string)
tf.Tensor(b'\nH\nF\n\x07comment\x12;\n9\n7 Two young , White males are outside near many bushes .', shape=(), dtype=string)
tf.Tensor(b'\nB\n@\n\x07comment\x125\n3\n1 Two men in green shirts are standing in a yard .', shape=(), dtype=string)
tf.Tensor(b'\n>\n<\n\x07comment\x121\n/\n- A man in a blue shirt standing in a garden .', shape=(), dtype=string)
tf.Tensor(b'\n9\n7\n\x07comment\x12,\n*\n( Two friends enjoy time spent together .', shape=(), dtype=string)
 Two young guys with shaggy hair look at their hands while hanging out in the yard .
 Two young , White males are outside near many bushes .
 Two men in green shirts are standing in a yard .
 A man in a blue shirt standing in a garden .
 Two friends enjoy time spent together .



[ 152  179  975  115 3875  383  551  103  325  591  186  954 

In [2]:
df = pd.read_csv('/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv', delimiter='|')
df.head(10)
import os
import pandas as pd
import tensorflow as tf

# Step 1: Reorder the files based on the result.csv file
# Define the path to the image folder and the result.csv file
image_folder = '/kaggle/input/flickr-image-dataset/flickr30k_images/flickr30k_images'
csv_file_path = '/kaggle/input/flickr-image-dataset/flickr30k_images/results.csv'

# Read the result.csv file
csv_data = pd.read_csv(csv_file_path, delimiter='|')

# Extract unique filenames from the CSV file
unique_filenames = csv_data['image_name'].unique().tolist()

# List all files in the directory
files_in_directory = os.listdir(image_folder)

# Reorder the files_in_directory based on unique_filenames
# Only include files that exist in both lists
reordered_files = [file for file in unique_filenames if file in files_in_directory]

full_file_paths = [os.path.join(image_folder, file) for file in reordered_files]

# Step 2: Integrate the reordered list into the TensorFlow dataset pipeline
def load_image(image_path):
    # Ensure the image path is a string
    image_path = tf.ensure_shape(image_path, [])
    
    # Read and process the image
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize for vision transformer (e.g., 224x224 for ViT)
    image = image / 255.0  # Normalize to [0, 1] range
    return image
# Create a TensorFlow dataset from the list of reordered image paths
dataset = tf.data.Dataset.from_tensor_slices(full_file_paths)

# Map the load_image function to preprocess the images
dataset = dataset.map(load_image)

# Batch and prefetch for performance
dataset = dataset.batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Inspect the dataset
for image in dataset.take(1):
    print(image.shape)  # Should be (batch_size, 224, 224, 3)
!pip install tensorflow tensorflow-hub
import tensorflow as tf
import tensorflow_hub as hub

# Load the pretrained Vision Transformer model from TensorFlow Hub
model = hub.KerasLayer("https://www.kaggle.com/models/spsayakpaul/vision-transformer/TensorFlow2/vit-b16-classification/1", trainable=False)
def load_dataset(image_paths):
    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(lambda x: tf.numpy_function(load_and_preprocess_image,[x], tf.float32))
    dataset = dataset.batch(32).prefetch(tf.data.AUTOTUNE)
    return dataset
def preprocess_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [224, 224])  # Resize to 224x224
    image = image / 255.0  # Normalize to [0, 1]
    return image
def load_and_preprocess_image(image_path):
    image = preprocess_image(image_path)
    return image
def get_embeddings(batch):
    embeddings = model(batch)
    return embeddings
# Create dataset
dataset = load_dataset(full_file_paths)

(32, 224, 224, 3)


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [3]:
# Generate embeddings for the entire dataset
i=0
image_embeddings = []
for batch in dataset:
    if i%100==0:
        print(i,end=" ")
    i+=1
    embeddings = get_embeddings(batch)
    image_embeddings.append(embeddings)
print(i)
image_embeddings = tf.concat(image_embeddings, axis=0)  
from tensorflow.keras.layers import Dense


0 

I0000 00:00:1725198383.264853      36 service.cc:145] XLA service 0x5ac27a2d31d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725198383.264903      36 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1725198383.264909      36 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1725198393.766382     234 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_6', 1300 bytes spill stores, 1284 bytes spill loads

I0000 00:00:1725198395.063680     233 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5', 1300 bytes spill stores, 1284 bytes spill loads

I0000 00:00:1725198397.605502     235 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_2', 1300 bytes spill stores, 1284 bytes spill loads

I0000 00:00:17

100 200 300 400 500 600 700 800 900 

I0000 00:00:1725199159.494951     273 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_5', 1300 bytes spill stores, 1284 bytes spill loads

I0000 00:00:1725199162.060712     274 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_109', 1296 bytes spill stores, 1280 bytes spill loads

I0000 00:00:1725199167.231738     272 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_6', 1300 bytes spill stores, 1284 bytes spill loads



994


In [4]:

output_dim = 768
dense_layer = Dense(output_dim, activation='relu')
reduced_embedding = dense_layer(image_embeddings)
print(reduced_embedding.shape)

(31783, 768)


In [30]:
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split

# Convert to NumPy array
h = common_embeddings
X = reduced_embedding


# concatenated_embeddings_np = np.concatenate([h, x], axis=-1)

# Perform train-test split
# h_train_np, h_val_np = train_test_split(h, test_size=0.2, random_state=42)

# X_train_np, X_val_np = train_test_split(x, test_size=0.2, random_state=42)

# # Convert to PyTorch tensor
# common_embeddings = torch.from_numpy(h)
# reduced_embedding = torch.from_numpy(x)
# print(type(reduced_embedding))

In [7]:
# import tensorflow as tf
# from tensorflow.keras.layers import Dropout

# class HopfieldLayer(tf.keras.layers.Layer):
#     def __init__(self, input_size, hidden_size, output_size, update_steps_max=3, beta=1.0, dropout_rate=0.3):
#         super(HopfieldLayer, self).__init__()
#         self.input_size = input_size
#         self.hidden_size = hidden_size
#         self.output_size = output_size
#         self.update_steps_max = update_steps_max
#         self.beta = beta
#         self.dropout_rate = dropout_rate

#         # Trainable weights with L2 regularization
#         self.W = self.add_weight(shape=(input_size, hidden_size), initializer="random_normal", trainable=True, regularizer=tf.keras.regularizers.L2(0.01))
#         self.V = self.add_weight(shape=(hidden_size, output_size), initializer="random_normal", trainable=True, regularizer=tf.keras.regularizers.L2(0.01))
#         self.U = self.add_weight(shape=(hidden_size, hidden_size), initializer="random_normal", trainable=True, regularizer=tf.keras.regularizers.L2(0.01))

#         # Dropout layer
#         self.dropout = Dropout(dropout_rate)

#     def energy_function(self, x):
#         """Compute the energy of the current state."""
#         energy = -0.5 * tf.matmul(tf.matmul(x, self.U), x, transpose_b=True)
#         return energy

#     def update_rule(self, x):
#         """Update rule for Hopfield dynamics."""
#         x_new = tf.nn.softmax(self.beta * tf.matmul(x, self.U))
#         return x_new

#     def call(self, inputs):
#         hidden = tf.matmul(inputs, self.W)
#         state = hidden

#         for _ in range(self.update_steps_max):
#             energy_before = self.energy_function(state)
#             state = self.update_rule(state)
#             energy_after = self.energy_function(state)

#             # Use TensorFlow operations to simulate conditional behavior
#             # Compare energy_before and energy_after
#             # (Note: This comparison should be designed based on your specific requirements)
#             # Here we use a fixed number of iterations as a fallback

#         outputs = tf.matmul(state, self.V)
#         outputs = self.dropout(outputs)  # Apply dropout
#         return outputs


In [25]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Define the Hopfield Layer
class HopfieldLayer(layers.Layer):
    def __init__(self, input_dim, hidden_dim):
        super(HopfieldLayer, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        
        # Linear transformation layers
        self.input_to_hidden = layers.Dense(hidden_dim, activation='tanh')
        self.hidden_to_hidden = layers.Dense(hidden_dim, activation='tanh')
        self.hidden_to_output = layers.Dense(input_dim)
        
        # Parameters for the Hopfield network (Energy function)
        self.weight_matrix = self.add_weight(shape=(hidden_dim, hidden_dim), initializer='random_normal', trainable=True)
        self.bias = self.add_weight(shape=(hidden_dim,), initializer='zeros', trainable=True)

    def call(self, x):
        # Initial hidden state (projection of the input)
        hidden_state = self.input_to_hidden(x)
        
        # Hopfield energy minimization process (fixed-point iteration)
        for _ in range(5):  # Number of iterations to reach convergence
            hidden_state = self.hidden_to_hidden(hidden_state) + tf.matmul(hidden_state, self.weight_matrix) + self.bias
        
        # Project the hidden state back to the output dimension
        output = self.hidden_to_output(hidden_state)
        return output

# Define the Self-Attention Layer
class SelfAttentionLayer(layers.Layer):
    def __init__(self, embed_dim, num_heads):
        super(SelfAttentionLayer, self).__init__()
        self.multihead_attn = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.layer_norm = layers.LayerNormalization(epsilon=1e-6)
        self.ffn = tf.keras.Sequential([
            layers.Dense(embed_dim * 4, activation='relu'),
            layers.Dense(embed_dim)
        ])

    def call(self, x):
        attn_output = self.multihead_attn(x, x)
        x = self.layer_norm(x + attn_output)  # Add & Norm
        ffn_output = self.ffn(x)
        output = self.layer_norm(x + ffn_output)  # Add & Norm
        return output

# Define the Text-to-Image Model
class TextToImageModel(Model):
    def __init__(self, text_dim, image_dim, hidden_dim, num_heads):
        super(TextToImageModel, self).__init__()
        
        # Embedding layers for text and image
        self.text_embedding = layers.Dense(hidden_dim)
        self.image_embedding = layers.Dense(hidden_dim)
        
        # Hopfield layer to integrate text and image embeddings
        self.hopfield_layer = HopfieldLayer(hidden_dim, hidden_dim)
        
        # Self-Attention layer after the Hopfield layer
        self.attention_layer = SelfAttentionLayer(hidden_dim, num_heads)
        
        # Decoder layer to generate the final image
        self.decoder = layers.Dense(image_dim)

    def call(self, text_embedding, image_embedding):
        # Project text and image embeddings to the same dimension
        text_embedded = self.text_embedding(text_embedding)
        image_embedded = self.image_embedding(image_embedding)
        
        # Combine the embeddings
        combined_embedding = text_embedded + image_embedded
        
        # Pass through the Hopfield layer
        hopfield_output = self.hopfield_layer(combined_embedding)
        
        # Reshape for multi-head attention (Assuming batch size as 1 for simplicity)
        hopfield_output = tf.expand_dims(hopfield_output, axis=1)  # Add sequence dimension
        
        # Pass through the Attention layer
        attention_output = self.attention_layer(hopfield_output)
        
        # Remove sequence dimension
        attention_output = tf.squeeze(attention_output, axis=1)
        
        # Generate the final image representation
        output_image = self.decoder(attention_output)
        return output_image

# Example usage
# text_dim = 768
# image_dim = 768
# hidden_dim = 256
# num_heads = 4

# # Example inputs (random for demonstration purposes)
# text_embedding = tf.random.normal([32, text_dim])
# image_embedding = tf.random.normal([32, image_dim])
text_dim = h.shape[1]
print(text_dim)
image_dim = X.shape[1]
print(image_dim)
hidden_dim = 1536
num_heads = 4


# Initialize the model
model = TextToImageModel(text_dim, image_dim, hidden_dim, num_heads)

# Forward pass
output_image = model(h, X)
print(output_image.shape)  # Should output: (32, 128)

768
768
(31783, 768)


In [31]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['accuracy']
)


In [32]:
# Convert numpy arrays back to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train_np, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val_np, dtype=tf.float32)
print(X_train.dtype)

<dtype: 'float32'>


In [34]:
from sklearn.model_selection import train_test_split

# Assuming concatenated_embeddings is your input data
X_train_images, X_val_images = train_test_split(x, test_size=0.2, random_state=42)

# Split text embeddings
X_train_texts, X_val_texts = train_test_split(h, test_size=0.2, random_state=42)


In [35]:
model.fit([X_train_images, X_train_texts], [X_train_images, X_train_texts],
          epochs=100,
          batch_size=32,
          validation_data=([X_val_images, X_val_texts], [X_val_images, X_val_texts]))

# Evaluation
eval_results = model.evaluate([X_val_images, X_val_texts], [X_val_images, X_val_texts])

Epoch 1/100


TypeError: missing a required argument: 'image_embedding'

In [ ]:
eval_results = mhn_model.evaluate(X_val, X_val)
print(f"Validation Loss: {eval_results[0]}")
print(f"Validation Accuracy: {eval_results[1]}")
# Example prediction
predictions = mhn_model.predict(X_val)


In [ ]:
# Generate images from some test embeddings
test_embeddings = X_val[:10]  # Just an example, use appropriate data
generated_images = mhn_model.predict(test_embeddings)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to plot images
def plot_images(images, ncols=5):
    n_images = images.shape[0]
    n_rows = (n_images + ncols - 1) // ncols
    
    fig, axs = plt.subplots(n_rows, ncols, figsize=(ncols * 2, n_rows * 2))
    axs = axs.flatten()
    
    for i in range(n_images):
        img = images[i]
        axs[i].imshow(img, cmap='gray')
        axs[i].axis('off')
    
    for j in range(i + 1, len(axs)):
        axs[j].axis('off')
    
    plt.tight_layout()
    plt.show()

# Generate images
test_embeddings = X_val[:10]  # Example, adjust as needed
generated_images = mhn_model.predict(test_embeddings)

# Assuming images are originally 32x32
image_height, image_width = 32, 32
reshaped_images = generated_images.reshape(-1, image_height, image_width)

# Display images
plot_images(reshaped_images)
